In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans

# Loads data.
dataset = spark.read.csv("seeds_dataset.csv",header=True,inferSchema=True)

In [3]:
dataset.head()

Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)

In [4]:
dataset.describe().show()

+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|summary|              area|         perimeter|         compactness|   length_of_kernel|   width_of_kernel|asymmetry_coefficient|   length_of_groove|
+-------+------------------+------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|  count|               210|               210|                 210|                210|               210|                  210|                210|
|   mean|14.847523809523816|14.559285714285718|  0.8709985714285714|  5.628533333333335| 3.258604761904762|   3.7001999999999997|  5.408071428571429|
| stddev|2.9096994306873647|1.3059587265640225|0.023629416583846364|0.44306347772644983|0.3777144449065867|   1.5035589702547392|0.49148049910240543|
|    min|             10.59|             12.41|              0.8081|              4.899|            

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [7]:
vec_assembler = VectorAssembler(inputCols = dataset.columns, outputCol='features')

In [8]:
final_data = vec_assembler.transform(dataset)

In [9]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [11]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [12]:
scalerModel = scaler.fit(final_data)

In [13]:
final_data = scalerModel.transform(final_data)

In [15]:
kmeans = KMeans(k=3, featuresCol="scaledFeatures")
model = kmeans.fit(final_data)

In [17]:
model.clusterCenters()

[array([ 4.06133795, 10.13721767, 35.82681204, 11.81771972,  7.5087187 ,
         3.25852121, 10.4215732 ]),
 array([ 6.3407095 , 12.39263108, 37.41143125, 13.92892299,  9.77251635,
         2.42396447, 12.28547936]),
 array([ 4.89953844, 10.91313605, 37.26599347, 12.37677589,  8.5736587 ,
         1.80144511, 10.362193  ])]

In [18]:
results = model.transform(final_data)

In [20]:
results.select("prediction").show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
+----------+
only showing top 20 rows



In [21]:
spark.stop()